In [10]:
pip install ortools


  Using cached ortools-9.3.10497-cp39-cp39-win_amd64.whl (40.5 MB)


In [11]:
"""Capacited Vehicles Routing Problem (CVRP)."""
import numpy as np
import pandas as pd
import random

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

from sklearn.neighbors import DistanceMetric
from math import radians

In [ ]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parents[0]))
from Helper_Functions import common_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "google_or_tools_paths", "urban_gb")

In [ ]:
data = pd.read_csv("urbanGB.csv")
data.head()

,latitude,longitude
0,-0.310990,53.7802
1,-3.112550,55.8706
2,-0.088533,51.3806
3,-0.326795,51.4476
4,-3.946530,51.6594


In [ ]:
data.shape

(360177, 2)

In [ ]:
df = data[0:1000]
df.shape

(1000, 2)

In [ ]:
# Convert the Lat/Long degress in Radians
df['latitude'] = np.radians(df['latitude'])
df['longitude'] = np.radians(df['longitude'])

# Create dist object with haversine metrics
dist = DistanceMetric.get_metric('haversine')

# we will use pairwise() function to calculate the
# haversine distance between each of the element with each other in this array
# Input to pairwise() function is numpy.ndarray
distance_matrix = dist.pairwise(df[['latitude','longitude']].to_numpy())*6373
distance_matrix = distance_matrix.tolist()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_distance_metric.py:14: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be 

In [ ]:
print(distance_matrix[2][3])

27.529712963952527


In [ ]:
type(distance_matrix)

list

In [ ]:
type(distance_matrix[:1])

list

In [ ]:
def create_data_model(matrix):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = matrix
    data['demands'] = [0]
    for i in range(1, 1000):
      data['demands'].append(random.randint(1, 10))
    total = 0
    for ele in range(0, len(data['demands'])):
      total = total + data['demands'][ele]
    capacity = int(total/10) + 1
    data['vehicle_capacities'] = [capacity] * 10
    data['num_vehicles'] = 10
    data['depot'] = 0

    data['time_windows'] = [(0, 5), (7, 12), (10, 15), (16, 18), (10, 13), (0, 5), (5, 10), (0, 4), (5, 10),
                            (0, 3), (10, 16), (10, 15), (0, 5), (5, 10), (7, 8), (10, 15), (11, 15)]
    print(f"Total Demand from 1000 locations: {total}")
    print(f"Total Capacity of all the vehicles: {capacity * 10}")
    return data

In [ ]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [ ]:
def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(distance_matrix)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 30

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    print(f"This is sol {solution}")
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Total Demand from 1000 locations: 5451
Total Capacity of all the vehicles: 5460
This is sol Assignment(Capacity0 (0) | Capacity1 (197) | Capacity2 (440) | Capacity3 (519) | Capacity4 (393) | Capacity5 (413) | Capacity6 (134) | Capacity7 (207) | Capacity8 (14) | Capacity9 (213) | Capacity10 (374) | Capacity11 (401) | Capacity12 (83) | Capacity13 (124) | Capacity14 (80) | Capacity15 (282) | Capacity16 (179) | Capacity17 (413) | Capacity18 (63) | Capacity19 (381) | Capacity20 (426) | Capacity21 (118) | Capacity22 (1) | Capacity23 (95) | Capacity24 (495) | Capacity25 (49) | Capacity26 (90) | Capacity27 (317) | Capacity28 (60) | Capacity29 (311) | Capacity30 (226) | Capacity31 (507) | Capacity32 (144) | Capacity33 (37) | Capacity34 (456) | Capacity35 (170) | Capacity36 (339) | Capacity37 (343) | Capacity38 (273) | Capacity39 (143) | Capacity40 (466) | Capacity41 (409) | Capacity42 (458) | Capacity43 (302) | Capacity44 (2) | Capacity45 (84) | Capacity46 (472) | Capacity47 (278) | Capacity48 

* The Haversine (or great circle) distance is the angular distance between two points on the surface of a sphere.
* The first coordinate of each point is assumed to be the latitude, the second is the longitude, given in radians
* SciPy is a scientific computation library that uses NumPy underneath. SciPy stands for Scientific Python
* It provides more utility functions for optimization, stats and signal processing.
* get_metrics() method of scipy class used to access the metrics
* 